In [1]:
import base64
import ssl
import requests
from urllib.parse import urljoin

from cryptography import x509
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.hashes import SHA1
from cryptography.x509 import ocsp
from cryptography.x509.ocsp import OCSPResponseStatus, OCSPCertStatus
from cryptography.x509.oid import ExtensionOID, AuthorityInformationAccessOID

In [2]:
def get_cert_for_hostname(hostname, port):
    conn = ssl.create_connection((hostname, port))
    context = ssl.SSLContext(ssl.PROTOCOL_SSLv23)
    sock = context.wrap_socket(conn, server_hostname=hostname)
    certDER = sock.getpeercert(True)
    certPEM = ssl.DER_cert_to_PEM_cert(certDER)
    return x509.load_pem_x509_certificate(certPEM.encode('ascii'), default_backend())

In [3]:
def get_cert_for_path(cert_path):
    with open(cert_path, "rb") as cert_file:
        certPEM = cert_file.read()
        return x509.load_pem_x509_certificate(certPEM, default_backend())

In [4]:
def get_issuer(cert):
    aia = cert.extensions.get_extension_for_oid(ExtensionOID.AUTHORITY_INFORMATION_ACCESS).value
    issuers = [ia for ia in aia if ia.access_method == AuthorityInformationAccessOID.CA_ISSUERS]
    if not issuers:
        raise Exception(f'no issuers entry in AIA')
    return issuers[0].access_location.value

In [5]:
def get_ocsp_server(cert):
    aia = cert.extensions.get_extension_for_oid(ExtensionOID.AUTHORITY_INFORMATION_ACCESS).value
    ocsps = [ia for ia in aia if ia.access_method == AuthorityInformationAccessOID.OCSP]
    if not ocsps:
        raise Exception(f'no ocsp server entry in AIA')
    return ocsps[0].access_location.value

In [6]:
def get_issuer_cert(ca_issuer):
    issuer_response = requests.get(ca_issuer)
    if issuer_response.ok:
        issuerDER = issuer_response.content
        issuerPEM = ssl.DER_cert_to_PEM_cert(issuerDER)
        return x509.load_pem_x509_certificate(issuerPEM.encode('ascii'), default_backend())
    raise Exception(f'fetching issuer cert  failed with response status: {issuer_response.status_code}')

In [7]:
def get_issuer_cert_path(cert_path):
    with open(cert_path, "rb") as cert_file:
        certPEM = cert_file.read()
        return x509.load_pem_x509_certificate(certPEM, default_backend())

In [8]:
def get_oscp_request(ocsp_server, cert, issuer_cert):
    builder = ocsp.OCSPRequestBuilder()
    builder = builder.add_certificate(cert, issuer_cert, SHA1())
    req = builder.build()
    req_path = base64.b64encode(req.public_bytes(serialization.Encoding.DER))
    return urljoin(ocsp_server + '/', req_path.decode('ascii'))

In [9]:
def get_ocsp_cert_status(ocsp_server, cert, issuer_cert):
    try:
        ocsp_resp = requests.get(get_oscp_request(ocsp_server, cert, issuer_cert))
    except:
        print(f"Can't connect to the OCSP server at {ocsp_server}.")
        return None
    if ocsp_resp.ok:
        ocsp_decoded = ocsp.load_der_ocsp_response(ocsp_resp.content)
        if ocsp_decoded.response_status == OCSPResponseStatus.SUCCESSFUL:
            return ocsp_decoded.certificate_status
        else:
            raise Exception(f'decoding ocsp response failed: {ocsp_decoded.response_status}')
    raise Exception(f'fetching ocsp cert status failed with response status: {ocsp_resp.status_code}')

In [10]:
def get_cert_status_for_host(hostname, port):
    print('   hostname:', hostname, "port:", port)
    cert = get_cert_for_hostname(hostname, port)
    with open("local_cert.pem", "wb") as f:
        f.write(cert.public_bytes(serialization.Encoding.PEM))
    ca_issuer = get_issuer(cert)
    print('   issuer ->', ca_issuer)
    issuer_cert = get_issuer_cert(ca_issuer)
    with open("issuer_cert.pem", "wb") as f:
        f.write(issuer_cert.public_bytes(serialization.Encoding.PEM))
    ocsp_server = get_ocsp_server(cert)
    print('   ocsp_server ->', ocsp_server)
    print('   ocsp_request ->', get_oscp_request(ocsp_server, cert, issuer_cert))
    return get_ocsp_cert_status(ocsp_server, cert, issuer_cert)

In [11]:
def get_cert_status_for_path(path, issuer_path):
    print('   path:', path)
    cert = get_cert_for_path(path)
    print(cert)
    issuer_cert = get_issuer_cert_path(issuer_path)
    ocsp_server = get_ocsp_server(cert)
    print('   ocsp_server ->', ocsp_server)
    print('   ocsp_request ->', get_oscp_request(ocsp_server, cert, issuer_cert))
    return get_ocsp_cert_status(ocsp_server, cert, issuer_cert)

In [12]:
get_cert_status_for_host("outlook.office.com", 443)

   hostname: outlook.office.com port: 443
   issuer -> http://cacerts.digicert.com/DigiCertCloudServicesCA-1.crt
   ocsp_server -> http://ocspx.digicert.com
   ocsp_request -> http://ocspx.digicert.com/MFEwTzBNMEswSTAJBgUrDgMCGgUABBRItqniEpOzwCCxKs5Oc2SaPGfcmwQU3VHQojFzqXOuj7QBfl2MV8uf8PcCEA8S3IlVgh1tk2vPNOUPYMU=


<OCSPCertStatus.GOOD: 0>

In [13]:
get_cert_status_for_path("CPO_cert_1.pem", "CPOSubCA_cert_1.pem")

   path: CPO_cert_1.pem
<Certificate(subject=<Name(C=US,O=Sandia,DC=CPO,OU=EV Department,CN=EV001)>, ...)>
   ocsp_server -> http://ocsp.demo.one.digicert.com
   ocsp_request -> http://ocsp.demo.one.digicert.com/MFUwUzBRME8wTTAJBgUrDgMCGgUABBQjNURBRu551spuO2JxTNQGRZOOlAQUMikgDWwe1Fgiq6IbpFSH8V2nZ50CFCoEYSyDT3ult45QRd8YsIS6f0dk


<OCSPCertStatus.GOOD: 0>

In [14]:
get_cert_status_for_path("local_cert.pem", "issuer_cert.pem")

   path: local_cert.pem
<Certificate(subject=<Name(C=US,ST=Washington,L=Redmond,O=Microsoft Corporation,CN=outlook.com)>, ...)>
   ocsp_server -> http://ocspx.digicert.com
   ocsp_request -> http://ocspx.digicert.com/MFEwTzBNMEswSTAJBgUrDgMCGgUABBRItqniEpOzwCCxKs5Oc2SaPGfcmwQU3VHQojFzqXOuj7QBfl2MV8uf8PcCEA8S3IlVgh1tk2vPNOUPYMU=


<OCSPCertStatus.GOOD: 0>

In [15]:
a = get_cert_status_for_path("CPOSubCA_cert_1.pem", "CPORootCA_cert_1.pem")

   path: CPOSubCA_cert_1.pem
<Certificate(subject=<Name(C=US,O=EV Charging PKI,DC=EVCPKI,OU=TEST CPO Sub-CA,CN=P-256 TEST Tier 1 CPO Sub-CA)>, ...)>
   ocsp_server -> http://ocsp.demo.one.digicert.com
   ocsp_request -> http://ocsp.demo.one.digicert.com/MFUwUzBRME8wTTAJBgUrDgMCGgUABBTQyWYYJgvYzCTgD1JGDg1fRnqIHAQU9eLCEldHooVf0lH1WfmUT4Mm6usCFHlV7Ka49kdpsodFEq1pF95Up1wX
